## 4-7. 프로젝트: 멋진 작사가 만들기


### Step 1. 데이터 다운로드

이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면, ~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.

### Step 2. 데이터 읽어오기

`glob` 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. `glob` 를 활용하여 모든 `txt` 파일을 읽어온 후, `raw_corpus` 리스트에 문장 단위로 저장하도록 할게요!

In [50]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['\ufeffbaby It was all a dream', 'I used to read Word Up magazine', 'Salt n Pepa and Heavy D up in the limousine']


### Step 3. 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

`preprocess_sentence()` 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [52]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

vacant = 0
colon = 0
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0:
        vacant += 1 # 정제된 갯수 파악
        continue
    if sentence[-1] == ":":
        colon += 1 # 정제된 갯수 파악
        continue    
    else:
        corpus.append(sentence)
        
# 확인
print("raw_corpus:", len(raw_corpus))
print("vacant    :", vacant)
print("colon     :", colon)
print("corpus    :", len(corpus))

raw_corpus: 187088
vacant    : 11102
colon     : 237
corpus    : 175749


In [54]:
# 패키지 불러오기
import re

# 함수 조금 변경
def preprocess_sentence(sentences):
    
    corpus = []
    # 문장 전체를 불러 오고
    for sentence in sentences:
        sentence = sentence.lower().strip()
        sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
        sentence = sentence.strip() # 모든 정제를 마친 후
        if len(sentence.split()) > 13:
            continue # 단어 13개 이상이면 건너 뛰기
        else:
            sentence = '<start> ' + sentence + ' <end>' # 6
            corpus.append(sentence)
    return corpus

In [55]:
# 단어 정제된 문장 리스트 받기
second_corpus = preprocess_sentence(corpus)
print(len(second_corpus))

156013


## Step 4. 평가 데이터셋 분리

#### 훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!



In [57]:
# 패키지 불러오기
import tensorflow as tf

# tokenize 함수 정의
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor, tokenizer)
    return tensor, tokenizer

In [58]:
# tokenize 및 텐서화
tensor, tokenizer = tokenize(second_corpus)

[[   2   52   11 ...    0    0    0]
 [   2    4  285 ...    0    0    0]
 [   2 2876  480 ...    0    0    0]
 ...
 [   2    8    4 ...    0    0    0]
 [   2   44   17 ...    0    0    0]
 [   2    6  172 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f24f454bb50>


In [66]:
tensor.shape

(156013, 15)

In [67]:
# 소스 문장, 타겟 문장 나누기
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

In [68]:
src_input[:3]

array([[    2,    52,    11,    53,    25,     9,   361,     3,     0,
            0,     0,     0,     0,     0],
       [    2,     4,   285,    10,   591,   483,    29,  2039,     3,
            0,     0,     0,     0,     0],
       [    2,  2876,   480, 12440,     8,  1099,   141,    29,    14,
            6,  4255,     3,     0,     0]], dtype=int32)

In [69]:
tgt_input[:3]

array([[   52,    11,    53,    25,     9,   361,     3,     0,     0,
            0,     0,     0,     0,     0],
       [    4,   285,    10,   591,   483,    29,  2039,     3,     0,
            0,     0,     0,     0,     0],
       [ 2876,   480, 12440,     8,  1099,   141,    29,    14,     6,
         4255,     3,     0,     0,     0]], dtype=int32)

In [70]:
# 패키지 불러오기
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         test_size=0.2,
                                                         random_state=42)

print(len(enc_train))


124810


In [71]:
# 데이터 갯수 확인
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


## Step 5. 인공지능 만들기


모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [72]:
# 데이터셋 준비
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [73]:
# 모델 설정
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    


### 모델 설정(1)
- embedding_size = 256
- hidden_size = 1024

In [74]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [75]:
# 모델 확인
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [76]:
# model.summary() 작동을 위해 한 배치 학습
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 14001), dtype=float32, numpy=
array([[[ 1.06408756e-04,  1.23805512e-04, -1.66697529e-04, ...,
         -1.05661493e-05,  2.16173707e-04,  7.13661357e-05],
        [-3.66049208e-05,  2.64188857e-04, -3.10873846e-04, ...,
          9.98007545e-06,  2.82292604e-04,  1.79583891e-04],
        [-3.06450762e-04,  2.85471295e-04, -5.21949027e-04, ...,
          1.80076109e-04,  4.47729341e-04,  3.70835915e-04],
        ...,
        [-4.03541955e-04,  8.98141152e-05, -1.62466709e-03, ...,
         -3.85716907e-04,  1.21071690e-03,  5.44760085e-04],
        [-7.83631840e-05,  3.13122116e-04, -2.08676327e-03, ...,
         -7.26928760e-04,  1.27460295e-03,  4.47473838e-04],
        [ 2.31234240e-04,  5.48413373e-04, -2.53360393e-03, ...,
         -1.02637359e-03,  1.32481242e-03,  3.39265011e-04]],

       [[ 1.06408756e-04,  1.23805512e-04, -1.66697529e-04, ...,
         -1.05661493e-05,  2.16173707e-04,  7.13661357e-05],
        [ 3.16377147e-04,  1.83287902e-05, -1

In [79]:
# 모델 확인
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3584256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  14351025  
Total params: 31,574,961
Trainable params: 31,574,961
Non-trainable params: 0
_________________________________________________________________


In [81]:
# 모델 컴파일 및 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
609/609 [==============================] - 224s 364ms/step - loss: 2.7786
Epoch 2/10
609/609 [==============================] - 222s 365ms/step - loss: 2.5930
Epoch 3/10
609/609 [==============================] - 222s 365ms/step - loss: 2.4262
Epoch 4/10
609/609 [==============================] - 222s 365ms/step - loss: 2.2628
Epoch 5/10
609/609 [==============================] - 224s 367ms/step - loss: 2.1216
Epoch 6/10
609/609 [==============================] - 223s 366ms/step - loss: 2.0006
Epoch 7/10
609/609 [==============================] - 223s 366ms/step - loss: 1.8892
Epoch 8/10
609/609 [==============================] - 223s 366ms/step - loss: 1.7948
Epoch 9/10
609/609 [==============================] - 223s 366ms/step - loss: 1.7092
Epoch 10/10
609/609 [==============================] - 222s 365ms/step - loss: 1.6246


In [82]:
# 예측 문장 출력하는 함수 정의
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: 
            break
        if test_tensor.shape[1] >= max_len: 
            break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [83]:
# 문장 생성!
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [84]:
# 문장 생성!
generate_text(model, tokenizer, init_sentence="<start> after", max_len=40)

'<start> after all the things that i been through <end> '

In [85]:
# 문장 생성!
generate_text(model, tokenizer, init_sentence="<start> how", max_len=40)

'<start> how many times i gotta tell that ass to come over ? <end> '